In [29]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-07-17; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-07-17; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [1]:
%serialconnect to --port="/dev/tty.usbserial-02EDKZTP" --baud=115200

# %serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200

# %serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200


Connecting to --port=/dev/tty.usbserial-02EDKZTP --baud=115200 
MicroPython v1.15-222-g8edc3aacd-dirty on 2021-07-17; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [5]:
from ulab import numpy as np
# import machine
# from machine import Pin

In [11]:
from ulab import scipy as spy

x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

sos = np.array([[ 2.33735320e-03,  1.14276495e-03,  2.33735320e-03,
         1.00000000e+00, -1.66007862e+00,  7.84212717e-01],
       [ 1.00000000e+00, -1.12157259e+00,  1.00000000e+00,
         1.00000000e+00, -1.56060243e+00,  8.47127269e-01],
       [ 1.00000000e+00, -1.99924978e+00,  1.00000000e+00,
         1.00000000e+00, -1.85367861e+00,  8.85737239e-01],
       [ 1.00000000e+00, -1.39634866e+00,  1.00000000e+00,
         1.00000000e+00, -1.53107779e+00,  9.32994854e-01],
       [ 1.00000000e+00, -1.99561284e+00,  1.00000000e+00,
         1.00000000e+00, -1.94893224e+00,  9.63308997e-01],
       [ 1.00000000e+00, -1.46127625e+00,  1.00000000e+00,
         1.00000000e+00, -1.53065680e+00,  9.82583511e-01],
       [ 1.00000000e+00, -1.99305833e+00,  1.00000000e+00,
         1.00000000e+00, -1.97781492e+00,  9.88377176e-01],
       [ 1.00000000e+00, -1.99207496e+00,  1.00000000e+00,
         1.00000000e+00, -1.98772843e+00,  9.97300457e-01]])

fs = 256
t = np.arange(0, 0.2*np.pi, 1/fs)
t_ds = t[::5] # downsample by factor of 5 -> 50Hz
f0 = 10

arg =t*2*np.pi
y = np.sin(arg*8)*0.5 + np.sin(arg*100)*0.9+np.sin(arg*1)*0.2+np.sin(arg*50)*0.7
y_true = np.sin(arg*8)*0.5
y_sos = spy.signal.sosfilt(sos, y)

In [2]:
%sendtofile lib/peripherals.py --source lib/peripherals.py

Sent 146 lines (4965 bytes) to lib/peripherals.py.


In [13]:
%sendtofile lib/core.py --source lib/core.py

Sent 96 lines (2804 bytes) to lib/core.py.


In [17]:
# see https://docs.micropython.org/en/latest/reference/constrained.html

import gc
import micropython

gc.enable()

gc.collect()
micropython.mem_info()
print('-----------------------------')
print('Initial free: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
def func():
    return
    with open('xy.json') as f:
        data = json.load(f)
        del data
gc.collect()
print('Func definition: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
func()
print('Func run free: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
gc.collect()
print('Garbage collect free: {} allocated: {}'.format(gc.mem_free(), gc.mem_alloc()))
print('-----------------------------')
# micropython.mem_info(1)

stack: 736 out of 15360
GC: total: 111168, used: 2448, free: 108720
 No. of 1-blocks: 15, 2-blocks: 8, max blk sz: 47, max free sz: 6614
-----------------------------
Initial free: 108720 allocated: 2448
Func definition: 108608 allocated: 2560
Func run free: 108480 allocated: 2688
Garbage collect free: 108608 allocated: 2560
-----------------------------


In [15]:
from lib.core import initialise, run
gc.collect()
initialise(offline=True)
gc.collect()

ADC initialised
SPI initialised
DigiPot set to minimum gain (1.8)


In [16]:
gc.collect()
run()

{12: 0.0463, 10: 0.0474, 7: 0.0515}
.{12: 0.0107, 10: 0.08450000000000001, 7: 0.0277}
{12: 0.0239, 10: 0.0356, 7: 0.0185}
.{12: 0.0359, 10: 0.0212, 7: 0.0396}
{12: 0.0268, 10: 0.0341, 7: 0.0253}
.{12: 0.0344, 10: 0.05609999999999999, 7: 0.0266}
{12: 0.0329, 10: 0.0235, 7: 0.0368}
.{12: 0.0204, 10: 0.0533, 7: 0.0319}
{12: 0.0395, 10: 0.0241, 7: 0.0246}
.{12: 0.0325, 10: 0.0357, 7: 0.0433}
{12: 0.07889999999999999, 10: 0.023, 7: 0.0155}
.{12: 0.03359999999999999, 10: 0.0394, 7: 0.0212}
{12: 0.0438, 10: 0.0832, 7: 0.0277}
.{12: 0.0304, 10: 0.07090000000000001, 7: 0.0231}
{12: 0.0338, 10: 0.07290000000000001, 7: 0.0279}
.{12: 0.04, 10: 0.0426, 7: 0.0151}
{12: 0.0263, 10: 0.05739999999999999, 7: 0.0228}
.{12: 0.0243, 10: 0.0274, 7: 0.0129}
{12: 0.0257, 10: 0.06109999999999999, 7: 0.0222}
.{12: 0.0183, 10: 0.0536, 7: 0.0344}
{12: 0.0205, 10: 0.0782, 7: 0.0366}
.{12: 0.0383, 10: 0.04, 7: 0.033}
{12: 0.0228, 10: 0.0659, 7: 0.0291}
.{12: 0.0455, 10: 0.0536, 7: 0.011}
{12: 0.0429, 10: 0.0515, 7:

In [59]:
import micropython
print(micropython.mem_info(1))

stack: 736 out of 15360
GC: total: 111168, used: 45344, free: 65824
 No. of 1-blocks: 389, 2-blocks: 87, max blk sz: 125, max free sz: 2250
GC memory layout; from 3ffe4db0:
00000: MDThhBMh=Dh==h=Thh=================Th=======h=======h=hh===h=Fh=
00400: h=h=h=====DFhMDSBMDhTh=========hMB..h...hLh===hLFhhLhhhhhLLMDShL
00800: FFh=hBhhLh======hh=====hLFhLhhh=hhLLhh=h==TLFFh===.h==.h====..h=
00c00: =...hS....h.hh=======.........h=======h========h=======h=======S
01000: h==========h==========h=======.............h====================
01400: h=.h===h.h=h.h=h.hS.h=h===h===h===hh===S.hhh.hh.hS.Fh===========
01800: =========hh====h.h=ShS...h==.h=h======h=======h==============Sh=
01c00: hhh=hh.hh.h==h.h====hhSh=======h==hh.h=hh==hh...D.BB.h=h........
02000: .h=h...................h=h....h=hh=======.h===h.h===h.h=h.hh....
02400: L.h==h=.h==h.........................h============h======h===h==
02800: ===h=====.....h===h=.h==h.h==h.h===h.h===h...MD.D...............
02c00: ..........................h=

In [36]:
from lib.scheduling import LedFlasher

green_led = Pin(5, Pin.OUT)
flasher = LedFlasher(0, 5, green_led)
flasher.start()

In [37]:
flasher.stop()

In [134]:
import urandom

def synth_x(f, Ns, noise_power=0.5, fs=250):
    """
    generate a synthetic signal vector
    
    args:
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    """
    t = np.arange(0, Ns/fs, 1/fs)
    return np.sin(t*2*np.pi*f)*(1+urandom.random()*noise_power)

def synth_X(f, Nc, Ns, noise_power=0.5, fs=200, f_std=0.02, noise_std=0.2):
    """
    Generate a matrix of several variations of the same target signal. This is used
    to simulate the measurement of a common signal over multiple EEG channels 
    that have different SNR characteristics.
    
    args:
    f [float]: target frequency of synthetic signal (Hz)
    Nc [int]: number of channels
    Ns [int]: number of samples (time samples)
    noise_power [float]: variance of WGN noise distribution
    fs [float]: sampling frequency (Hz)
    f_std [float]: standard dev. of freq. in generated signal across channels to simulate interference from other frequency components over different channels
    noise_std [float]: standard dev. of noise across channels
    """
    X = []
    for i in range(Nc): # simulate noisy sinusoids with varying SNR across Nc channels
        f_i = f*(1+urandom.random()*0.1)
        x = synth_x(f_i, Ns, noise_power=0)

        X.append(x)
        
    return np.array(X)

def harmonic_reference(f0, fs, Ns, Nh=2, standardise_out=False):
    
    '''
    Generate reference signals for canonical correlation analysis (CCA)
    -based steady-state visual evoked potentials (SSVEPs) detection [1, 2].
    function [ y_ref ] = cca_reference(listFreq, fs,  Ns, Nh) 
    Input:
      f0        : stimulus frequency
      fs              : Sampling frequency
      Ns              : # of samples in trial
      Nh          : # of harmonics
    Output:
      y_ref           : Generated reference signals with shape (Nf, Ns, 2*Nh)
    '''  

    tidx = np.arange(1,Ns+1)*(1/fs) #time index
    
    tmp = []
    for harm_i in range(1,Nh+1):
        # Sin and Cos
        s = np.sin(tidx*2*np.pi*harm_i*f0)
        tmp.extend([s,
                    np.cos(tidx*2*np.pi*harm_i*f0)])
    y_ref = np.array(tmp)
    if standardise_out: # zero mean, unit std. dev
        return standardise(y_ref)
    return y_ref

In [187]:
stim_freqs = [7, 10]
fs = 250

gc.collect()

def tmp():
    x = synth_x(10, 200)
    x = x.reshape((1, len(x)))

    cca = CCA(stim_freqs, fs)
    result = cca.compute_corr(x)
    return result

result = tmp()
print(result)

{10: array([1.0], dtype=float64), 7: array([0.1521154798735567], dtype=float64)}


In [182]:
x = [3,4,5]
x = np.array(x).reshape((1, len(x)))
print(x)

array([[3.0, 4.0, 5.0]], dtype=float64)


In [20]:
def solve_qr(A, iterations=30):

    Ak = A
    Q_bar = np.eye(len(Ak))

    for k in range(iterations):
        Qk, Rk = np.linalg.qr(Ak)
        Ak = np.dot(Rk, Qk)
        Q_bar = np.dot(Q_bar, Qk)

    lam = np.diag(Ak)
    return lam, Q_bar

In [95]:
import gc

gc.collect()

Ns = 50

X = synth_X(7, 4, Ns, noise_power=0.2, f_std=0.04)
Y = harmonic_reference(12, 250, Ns, Nh=1)

# X = X.T
# Y = Y.T

print(X.shape(), Y.shape())

(4, 50) (2, 50)


In [18]:
Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
M2 = np.dot(np.linalg.inv(Cyy), Cyx)
M = np.dot(M1, M2)

In [21]:
lam, V = solve_qr(M, iterations=100)
print(np.sqrt(lam))

array([0.2094223883303393, 0.09026236148649224, 0.05508118227979756, 0.004485854551801153], dtype=float64)


In [108]:
import ujson as json

with open('data.json', 'w') as jsonfile:
    json.dump(X, jsonfile)

In [6]:
import ujson as json

with open('xy.json') as f:
    data = json.load(f)

In [11]:
X = np.array(data['X'])
Y = np.array(data['Y'])

In [117]:
from lib.computation import solve_eig_qr

lam, V = solve_eig_qr(np.dot(A, A.transpose()), 2)
print(lam, V[:, 0])

array([3090257647770.855, 1.84513789676072, 0.690618760063498, -6.033914926509784e-05], dtype=float64) array([-0.02983780014940622, -0.7932272635549372, 0.5364013569551137, 0.2866597256192484], dtype=float64)


In [105]:
Q, R = np.linalg.qr(A)
print(R)

array([[-883550.4911050206, -877556.5360877957, -878760.5076758521, -875939.8801019968],
       [-1.844490843821283e-11, -1.540692204816151, -1.705187238642372, -1.081242480290623],
       [-2.640966340204573e-11, 0.0, 0.4207967891230313, -0.778845463474262],
       [-5.666316844096295e-11, -0.0, 0.0, 1.354608353265618e-08]], dtype=float64)


In [45]:
X = np.dot(A, A.transpose())
print(np.dot(X, V[0]))
print(V[0]*lam[0])

array([-2.643192e+10, -7.02683e+11, 4.751729e+11, 2.539385e+11], dtype=float32)
array([-26136.41, -650175.6, 644677.9, 3.090257e+12], dtype=float32)


In [118]:
#TODO: uLab QR seems to give incorrect result for this matrix
A_null = np.array([[-3.090257e+12, -7.008567e+05,  4.739371e+05,  2.532784e+05],
       [-2.618467e+04, -3.090258e+12,  4.707228e+05,  2.515600e+05],
       [-2.622055e+04, -6.970566e+05, -3.090256e+12,  2.519048e+05],
       [-2.613640e+04, -6.948194e+05,  4.698551e+05, -3.090257e+12]])

In [119]:
Q, R = np.linalg.qr(A_null, mode="reduced")
print(R)

array([[3090257000000.001, 727041.3902643195, -447716.5664473673, -227142.0042689324],
       [-1.04984705117826e-25, 3090258000000.152, 226333.2414550781, 443259.1143798828],
       [2.568302040251067e-12, -0.0, -3090256000000.068, 721760.0402832033],
       [-2.576570284484911e-12, 0.0, -0.000244140625, -3090256999999.907]], dtype=float64)


In [129]:
from lib.decoding import harmonic_reference, CCA
from lib.computation import solve_gen_eig_prob
from ulab import numpy as np

gc.collect()

f0 = 7
stim_freqs = [7, 10, 12]
fs = 200

X_test = synth_X(10, 4, 10)

cca = CCA(stim_freqs, fs)

print(cca.compute_corr(X_test))


{12: nan, 10: nan, 7: nan}


In [144]:
print(Y)

array([[-2.171207463750402, -1.338522784901037, -0.5880826741808868, 0.04396681273567864, 0.5271820793288486, 0.8382883489982348, 0.9623007290342446, 0.8932459800949776, 0.6344502260963639, 0.1983787465439827],
       [1.305341814301688, 1.170640910201912, 0.9533601308922323, 0.6639651258411464, 0.3163950326494211, -0.07260892208832805, -0.484309788674666, -0.8988773871495461, -1.296343457179863, -1.657563458793997],
       [0.267963264843056, 0.7796799141142366, 1.073691207707363, 1.094033302783363, 0.8368341650474374, 0.3510505945826392, -0.2708505000816015, -0.9104927984376731, -1.446123014681661, -1.77578613587716],
       [1.850696116855231, 1.432374314847385, 0.8242438445293502, 0.142059844591912, -0.4843267608957941, -0.9356858528536302, -1.12610308092582, -1.019333307398602, -0.6356997203802178, -0.04822539836981504]], dtype=float64)


In [138]:
from lib.decoding import harmonic_reference

X = X_test
Y = harmonic_reference(7, 200, np.max(X_test.shape()), Nh=2, standardise_out=True)

Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

M1 = np.dot(np.linalg.inv(Cxx), Cxy) # intermediate result
M2 = np.dot(np.linalg.inv(Cyy), Cyx)

lam, _ = max_eig(np.dot(M1, M2), 20)

In [151]:
print(np.linalg.inv(Cxx))

array([[597636288720971.6, -590244015165738.0, -70701016851736.37, 63307345255390.93],
       [-387237973120798.1, 382448155728474.1, 45810658901710.62, -41019935647656.07],
       [-46384381598032.41, 45810634090902.22, 5487518240656.996, -4913662265777.822],
       [41533661101626.68, -41019913615534.64, -4913662287824.636, 4399817679136.317]], dtype=float64)


In [20]:
def zeros_like(A):
    return np.zeros(A.shape)

def block_diag(X, Y, reverse=False):
    if not reverse:
        X = np.concatenate((X, zeros_like(X)), axis=1)
        Y = np.concatenate((zeros_like(Y), Y), axis=1)
    else:
        X = np.concatenate((zeros_like(X), X), axis=1)
        Y = np.concatenate((Y, zeros_like(Y)), axis=1)
    return np.concatenate((X, Y), axis=0)

X = np.array([[0.0, 0.2552531, 0.4935954, 0.6992362, 0.8585516, 0.9609866, 0.9997549, 0.972288, 0.8804055, 0.7301948],
       [0.0, 0.2651061, 0.5112409, 0.7207904, 0.8787591, 0.9738424, 0.9992362, 0.9531231, 0.838803, 0.6644569],
       [0.0, 0.2634635, 0.5083104, 0.7172394, 0.8754874, 0.9718722, 0.9995833, 0.9566626, 0.8461428, 0.6758333],
       [0.0, 0.2671577, 0.5148946, 0.7252015, 0.8827904, 0.9762053, 0.9986557, 0.9485094, 0.8294118, 0.6500207]])

# Y = cca_reference([7], 200, 10, Nh=2)

Y = np.array([[-2.171207, -1.338523, -0.5880827, 0.04396701, 0.5271821, 0.8382883, 0.9623005, 0.8932458, 0.6344502, 0.1983788],
       [1.305342, 1.170641, 0.9533603, 0.6639652, 0.3163952, -0.07260892, -0.4843098, -0.8988775, -1.296343, -1.657563],
       [0.2679634, 0.7796801, 1.073691, 1.094033, 0.8368343, 0.3510505, -0.2708505, -0.9104931, -1.446123, -1.775786],
       [1.850696, 1.432374, 0.8242437, 0.1420597, -0.4843266, -0.9356859, -1.126103, -1.019333, -0.6356997, -0.04822552]])

In [38]:
from lib.computation import solve_gen_eig_prob

Cxx = np.dot(X, X.transpose()) # auto correlation matrix
Cyy = np.dot(Y, Y.transpose()) 
Cxy = np.dot(X, Y.transpose()) # cross correlation matrix
Cyx = np.dot(Y, X.transpose()) # same as Cxy.T

A = block_diag(Cxy, Cyx, reverse=True)
B = block_diag(Cxx, Cyy)

lam, Phi = solve_gen_eig_prob(A, B, eps=1e-6)

Traceback (most recent call last):
  File "<stdin>", line 11, in <module>
  File "computation.py", line 23, in solve_gen_eig_prob
ValueError: input matrix is asymmetric


In [41]:
from lib.computation import solve_eig_qr

print(A)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ImportError: can't import name solve_eig_qr


In [37]:
eps = 1e-6

def replace_nan(A, rep=0):
    return np.where(np.isfinite(A), A, rep)

Lam_b, Phi_b = np.linalg.eig(B) # eig decomp of B alone
Lam_b = np.eye(len(Lam_b))*Lam_b # convert to diagonal matrix of eig vals

Lam_b_sq = replace_nan(Lam_b**0.5)+np.eye(len(Lam_b))*eps
print(np.linalg.inv(Lam_b_sq))

array([[0.2104852, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1267.649, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 12.49017, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 1000000.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 8.926649, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.873826, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3365085, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1799781]], dtype=float32)


In [109]:
%ls

Listing directory '/'.
     3117    .env
      139    boot.py
      487    data.json
             lib/


In [50]:
%ls --recursive lib

Listing directory 'lib'.
       27    lib/__init__.mpy
     1643    lib/computation.mpy
      558    lib/config.mpy
     1065    lib/core.mpy
      722    lib/decoding.mpy
     8703    lib/mqtt_as.mpy
      790    lib/networking.mpy
     2504    lib/peripherals.mpy
     1052    lib/scheduling.mpy
     1174    lib/signal.mpy
     2601    lib/umqtt.mpy
      614    lib/utils.mpy
      641    lib/websockets.mpy


In [214]:
%sendtofile --source lib/.env .env  --binary

Sent 3118 bytes in 104 chunks to .env.


In [195]:
%sendtofile lib/core.py --source lib/core.py

Sent 83 lines (2491 bytes) to lib/core.py.


In [131]:
%sendtofile lib/decoding.py --source lib/decoding.py

Sent 61 lines (2106 bytes) to lib/decoding.py.


In [191]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-07-17; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.15-222-g8edc3aacd-dirty on 2021-07-17; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [111]:
%fetchfile data.json --print

Fetched 487=487 bytes from data.json.


In [20]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute] [--source [SOURCE]] [--quiet]
                   [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US